In [1]:
import setup
setup.init_django()

0pEnqPfhRk7bYu5LSa7ppMuKfqr21kck


In [2]:
from market.models import StockQuote

In [3]:
from django.db.models import Avg, F, RowRange, Window
from django.utils import timezone
from datetime import timedelta
from decimal import Decimal

In [4]:
days_ago = 30
now = timezone.now()
start_date = now - timedelta(days=30)
end_date = now

qs = StockQuote.objects.filter(time__range=(start_date, end_date))
qs.count()

15092

In [5]:
total = 0
for obj in qs:
    total += obj.close_price

In [6]:
total / qs.count()

Decimal('134.9934893586005830903790087')

In [15]:
qs.values('company').annotate(avg_price=Avg('close_price'))

<QuerySet [{'company': 12, 'avg_price': Decimal('138.556142439148')}]>

In [16]:
count = 5
ticker = "NVDA"
rolling_qs = list(qs.filter(company__ticker=ticker).order_by('-time')[:count])
rolling_qs.reverse()

In [17]:
rolling_qs

[<StockQuote: StockQuote object (705734)>,
 <StockQuote: StockQuote object (705735)>,
 <StockQuote: StockQuote object (705736)>,
 <StockQuote: StockQuote object (705737)>,
 <StockQuote: StockQuote object (705738)>]

In [18]:
total = 0
for i, obj in enumerate(rolling_qs):
    total += obj.close_price
    avg = total / (i + 1)
    print(i + 1, obj.id, obj.close_price, avg)

1 705734 141.1900 141.1900
2 705735 141.2300 141.2100
3 705736 141.7000 141.3733333333333333333333333
4 705737 142.8896 141.7524
5 705738 142.8800 141.97792


In [19]:
qs = StockQuote.objects.filter(company__ticker=ticker, time__range=(start_date, end_date))

In [20]:
frame_start = -(count - 1)
ma_val = qs.annotate(
        ma=Window(
            expression=Avg('close_price'),
            order_by=F('time').asc(),
            partition_by=[],
            frame=RowRange(start=frame_start, end=0),
        )
).order_by('-time')

In [23]:
for obj in ma_val[:20]:
    print(obj.id, obj.close_price, obj.ma, obj.time)

705738 142.8800 141.977920000000 2025-01-21 22:30:00+00:00
705737 142.8896 141.647920000000 2025-01-21 22:25:00+00:00
705736 141.7000 141.342000000000 2025-01-21 22:20:00+00:00
705735 141.2300 141.263760000000 2025-01-21 22:15:00+00:00
705734 141.1900 141.277760000000 2025-01-21 22:10:00+00:00
705733 141.2300 141.299760000000 2025-01-21 22:05:00+00:00
705732 141.3600 141.313760000000 2025-01-21 22:00:00+00:00
705731 141.3088 141.301760000000 2025-01-21 21:55:00+00:00
705730 141.3000 141.276000000000 2025-01-21 21:50:00+00:00
705729 141.3000 141.226120000000 2025-01-21 21:45:00+00:00
705728 141.3000 141.196120000000 2025-01-21 21:40:00+00:00
705727 141.3000 141.140120000000 2025-01-21 21:35:00+00:00
705726 141.1800 141.082120000000 2025-01-21 21:30:00+00:00
705725 141.0506 141.018160000000 2025-01-21 21:25:00+00:00
705724 141.1500 140.998040000000 2025-01-21 21:20:00+00:00
705723 141.0200 140.934040000000 2025-01-21 21:15:00+00:00
705722 141.0100 140.956040000000 2025-01-21 21:10:00+00:

In [22]:
frame_start = -(count - 1)
ma_vals = qs.annotate(
        ma_5=Window(
            expression=Avg('close_price'),
            order_by=F('time').asc(),
            partition_by=[],
            frame=RowRange(start=-4, end=0),
        ),
        ma_20=Window(
            expression=Avg('close_price'),
            order_by=F('time').asc(),
            partition_by=[],
            frame=RowRange(start=-19, end=0),
        )
).order_by('-time').first()

ma_vals.id, ma_vals.close_price, ma_vals.ma_5, ma_vals.ma_20

(705738,
 Decimal('142.8800'),
 Decimal('141.977920000000'),
 Decimal('141.351960000000'))